- 2021년 1월부터 6월까지 서담의 텍스트를 워드클라우드 시각화
- stylecloud를 활용해서 원하는 모양으로 워드클라우드 제작

# 라이브러리 및 파일 불러오기 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from collections import Counter
from wordcloud import WordCloud
import stylecloud
import urllib.request
from konlpy.tag import Okt
okt = Okt()

plt.rc('font', family='Malgun Gothic') #Windows
plt.rc("axes", unicode_minus=False) #마이너스 부호 표시

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina") #폰트 선명하게 표시

C:\Users\park3\AppData\Local\Temp/ipykernel_18976/2975013300.py:19: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("retina") #폰트 선명하게 표시


In [2]:
# 토큰화한 데이터프레임 파일 불러오기
ssodam = pd.read_csv('./data_l/ssodam2021half+popularity_tokenized.csv')

# 전처리

## 결측지 제거 

In [3]:
#결측치 제거
#thresh: optional 변수로 nan이 아닌 데이터가 몇 개면 삭제하지 않을 것인지를 정하는 변수
ssodam = ssodam.dropna(thresh = 7)
#한글, 숫자, %를 제외하고 지움
ssodam['text'] = ssodam['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣, \d, % ]","")

C:\Users\park3\AppData\Local\Temp/ipykernel_18976/3002474134.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  ssodam['text'] = ssodam['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣, \d, % ]","")


## 부적절한 글 제거 

In [4]:
#도배글 확인 후 제거 ex. 떡볶이
idx = ssodam[ssodam['num'] == 1156856].index
ssodam.drop(idx, inplace=True)

## 문자열을 리스트로 변환

In [5]:
def get_noun_list_from_str(msg):
    import re
    try :
        return [tuple([re.sub("'",'',y) for y in re.findall('\'.*?\'',x)]) for x in re.findall('\(.*?\)',msg)]
    except :
        return np.NaN

In [6]:
ssodam = ssodam.fillna('')
ssodam.text_tokenized = ssodam.text_tokenized.apply(lambda x: get_noun_list_from_str(x))
ssodam.title_tokenized = ssodam.title_tokenized.apply(lambda x: get_noun_list_from_str(x))
ssodam.comments_tokenized = ssodam.comments_tokenized.apply(lambda x: get_noun_list_from_str(x))
ssodam.comments_writer_tokenized = ssodam.comments_writer_tokenized.apply(lambda x: get_noun_list_from_str(x))
ssodam.upload_date = pd.to_datetime(ssodam.upload_date)

## 불용어 제거 

In [7]:
examples = pd.read_csv('./data/stopwords_.txt', encoding='UTF-8', header=None)
examples_lst = examples[0].values.tolist()

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','을','안','한','로',',','아','다미','부터','이랑',
             '자','에','와','하다','ㅋ','명','것','곳','거','때','그','데','못','중','대한','인데','진짜','너무','에서','근데',
             '000', '원', '데헷']

extra = '캡쳐 및 복사 금지 해당 게시물은 서강대 동문만을 위해 작성된 게시물로 외부로 유출시 서담 영구 탈퇴되며 저작권 침해 개인정보보호법 위반 등의 법적 책임을 질 수 있습니다'
extra_lst1 = okt.morphs(extra)
extra_lst2 = extra.split() #보조장치

stopwords += extra_lst1
stopwords += extra_lst2
stopwords += examples_lst
stopwords = list(set(stopwords))

In [11]:
ssodam['text_tokenized'][0]

[('2021년', 'Number'),
 ('은', 'Foreign'),
 ('모두', 'Noun'),
 ('행복한', 'Adjective'),
 ('한해', 'Noun'),
 ('가', 'Josa'),
 ('되길', 'Verb')]

In [8]:
ssodam.text_tokenized = tqdm(ssodam.text_tokenized.apply(lambda x: [item for item in x if item[0] not in stopwords]))
ssodam.title_tokenized = tqdm(ssodam.title_tokenized.apply(lambda x: [item for item in x if item[0] not in stopwords]))
ssodam.comments_tokenized = tqdm(ssodam.comments_tokenized.apply(lambda x: [item for item in x if item[0] not in stopwords]))
ssodam.comments_writer_tokenized = tqdm(ssodam.comments_writer_tokenized.apply(lambda x: [item for item in x if item[0] not in stopwords]))

100%|███████████████████████████████████████████████████████████████████████| 77979/77979 [00:00<00:00, 1182204.92it/s]
C:\Users\park3\anaconda3\envs\data\lib\site-packages\pandas\core\common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
100%|████████████████████████████████████████████████████████████████████████| 77979/77979 [00:00<00:00, 886688.95it/s]


## 단어 묶기 

In [9]:
# 앞 뒤 토큰을 묶어서 합칠 수 있으면 하나의 단어로 묶어서 인식시키는 함수 (ex. 서강 + 대학교)
def concat(x):    
    w = []
    for i in range(len(x)):
        if x[i][1] == 'Noun' or x[i][1] == 'Number':
            try:
                if x[i-1][1] == 'Modifier':
                    pass
                else:
                    w.append(x[i][0])
            except:
                pass
        elif x[i][1] == 'Suffix':
            try:
                w.pop()
                w.append(x[i-1][0] + x[i][0])
            except:
                pass
        elif x[i][1] == 'Modifier':
            try:
                w.append(x[i][0] + x[i+1][0])
            except:
                pass
    return w

In [10]:
tqdm.pandas() #진행률 프로세스바
ssodam['text_w'] = ssodam.text_tokenized.progress_apply(lambda x: concat(x))
ssodam['title_w'] = ssodam.title_tokenized.progress_apply(lambda x: concat(x))

100%|█████████████████████████████████████████████████████████████████████████| 77979/77979 [00:00<00:00, 79619.03it/s]


# Stylecloud 시각화 

In [11]:
stopwords_ = pd.read_csv('./data/stopwords__.txt', encoding='UTF-8', header=None)
stopwords_ = set(stopwords_[0].values.tolist())
stopwords__ = {'부연위원','드릴예정','충성','더리움','생각함','2외국어','구성위','차촬영','서강49','료지원','차면접','위원19년','심폐지','폐지구력','달정도' '부연위원','연구원부연','차합격자','생인턴','서강학','관심분','분요','성바이오피스','톡실행','아시분','니사례','녹화구합','애', '구합니', '기보본강의', '양도합도합니', '대해공부', '학년1', '점만점', '알도움', '공유분','차시험', '이기때문', '츠벤또', '세입문', '업종마카', '맞이사업', '맞이단', '새내기위백', '카페21', '컴공복'}
stopwords_ = stopwords_.union(stopwords__)
stopwords_

{'2외국어',
 '공부분',
 '공유구합',
 '공유분',
 '관심분',
 '구성위',
 '구합니',
 '그냥내',
 '급공채반',
 '급준비',
 '기보본강의',
 '기본본강의',
 '내더',
 '녹화구합',
 '니사례',
 '단2021',
 '달정도',
 '달정도부연위원',
 '대해공부',
 '더내',
 '더리움',
 '드릴예정',
 '료지원',
 '말내',
 '맞이단',
 '맞이사업',
 '문제풀',
 '방법대해',
 '법률널',
 '부연위원',
 '분구합',
 '분구해',
 '분요',
 '사람더',
 '사업입니다백',
 '새내기위백',
 '생각사람',
 '생각함',
 '생인턴',
 '서강49',
 '서강학',
 '성바이오피스',
 '세입문',
 '심폐지',
 '아시분',
 '알도움',
 '애',
 '양도합도합니',
 '양로2',
 '업종마카',
 '연구원부연',
 '위원19년',
 '이기때문',
 '점만점',
 '준비분',
 '지금내',
 '차공부',
 '차과목',
 '차면접',
 '차시험',
 '차준비',
 '차촬영',
 '차합격',
 '차합격자',
 '충성',
 '츠벤또',
 '친구내',
 '카페21',
 '컴공복',
 '톡실행',
 '파이토',
 '폐지구력',
 '학년1'}

## Color 선정

In [40]:
def _style_cloud(board, icon):
    '''for color palette test'''
    if board == '새내기게시판':
        target = ssodam[(ssodam['board'] == board) |
                        (ssodam['board'] == '새내기정보')]
    else:
        target = ssodam[ssodam['board'] == board]
    tokenized_text = [
        list(zip((sentence), (sentence)[1:]))
        for sentence in target.text_w.values
    ]  # text_W 토큰화
    
    tokenized_title = [
        list(zip((sentence), (sentence)[1:]))
        for sentence in target.title_w.values
    ]  # title_w 토큰화해서 합치기
    
    tokenized_nouns = tokenized_text + tokenized_title

    counter = Counter()
    for i in tokenized_nouns:
        counter.update(Counter(i))
    mc = counter.most_common()

    wdic = {}
    for i in mc:
        if i[0][0] != i[0][1]:
            wdic[i[0][0] + i[0][1]] = i[1]
        else:
            wdic[i[0][0]] = i[1]
    
    color_palletes = ['mycarta.LinearL_10',
                      'lightbartlein.diverging.BrownBlue12_8',
                      'lightbartlein.sequential.Blues7_7',
                      'colorbrewer.diverging.RdBu_4',
                      'colorbrewer.sequential.Blues_5',
                      'colorbrewer.sequential.BuGn_5',
                      'colorbrewer.sequential.BuPu_6',
                      'colorbrewer.sequential.GnBu_7',
                      'colorbrewer.sequential.YlGnBu_8',
                      'cmocean.diverging.Balance_18',
                      'cmocean.sequential.Dense_13']
    
    for ith, pallete in enumerate(color_palletes):
        if '/' in board:
            board = board.split('/')
            fname = './WordCloud/fig' + board[0] + '_colortest' + str(ith) + '.png'
        else:
            fname = './WordCloud/fig' + str(board) + '_colortest' + str(ith) + '.png'

        stylecloud.gen_stylecloud(text=wdic,
                                  icon_name=icon,
                                  font_path='NanumSquareR.ttf',
                                  palette=pallete,
                                  background_color='#F9FCFD',
                                  gradient="horizontal",
                                  custom_stopwords=stopwords_,
                                  output_name=fname)

    return

In [41]:
_style_cloud("익게2", "fas fa-user-friends")

## Cloud 생성

In [17]:
def style_cloud(board, icon):
    if board == '새내기게시판':
        target = ssodam[(ssodam['board'] == board) |
                        (ssodam['board'] == '새내기정보')]
    else:
        target = ssodam[ssodam['board'] == board]
    tokenized_text = [
        list(zip((sentence), (sentence)[1:]))
        for sentence in target.text_w.values
    ]  # text_W 토큰화
    
    tokenized_title = [
        list(zip((sentence), (sentence)[1:]))
        for sentence in target.title_w.values
    ]  # title_w 토큰화해서 합치기
    
    tokenized_nouns = tokenized_text + tokenized_title

    counter = Counter()
    for i in tokenized_nouns:
        counter.update(Counter(i))
    mc = counter.most_common()

    wdic = {}
    for i in mc:
        if i[0][0] != i[0][1]:
            wdic[i[0][0] + i[0][1]] = i[1]
        else:
            wdic[i[0][0]] = i[1]

    if '/' in board:
        board = board.split('/')
        fname = './WordCloud/fig' + board[0] + '.png'
    else:
        fname = './WordCloud/fig' + str(board) + '.png'
        
    stylecloud.gen_stylecloud(text=wdic,
                              icon_name=icon,
                              font_path='NanumSquareR.ttf',
                              palette="cmocean.diverging.Balance_19",
                              background_color='#F9FCFD',
                              gradient="horizontal",
                              custom_stopwords=stopwords_,
                              output_name=fname)

    return

- 폰트 색깔

https://jiffyclub.github.io/palettable/

In [45]:
style_cloud("익게2", "fas fa-user-friends")
style_cloud('연애상담소', "fas fa-heart")
style_cloud("CPA", "fas fa-calculator")
style_cloud("고시/전문직", "fas fa-book-open")
style_cloud("로스쿨", "fas fa-balance-scale")
#style_cloud("맛집", "fas fa-utensils") - 34개의 게시물밖에 없음
style_cloud("새내기게시판", "fas fa-grin")
style_cloud("취업게시판", "fas fa-user-tie")
style_cloud("자유게시판", "fas fa-users")
style_cloud("헬스", "fas fa-dumbbell")
style_cloud("졸업생", "fas fa-graduation-cap")
style_cloud("강의평가", "fas fa-pen")
style_cloud("익게1", "fas fa-user")
style_cloud("총학생회", "fas fa-university")
#style_cloud("서강학보", "fab fa-readme")
style_cloud("주식/투자", "fas fa-poll")
#style_cloud("서르비", "fas fa-question")
style_cloud("일반대학원", "fas fa-book-reader")

# 참고) 검색도구

키워드와 월을 이용해서 게시글을 검색합니다.

In [14]:
def search_cd(word, x):
    try:
        return True if word in [y[0] for y in x] else False
    except:
        return False

def s_search(word, month, show_urls=True):
    ssodam_month = ssodam[ssodam.upload_date.apply(lambda x: True if x.month==month else False)]
    title_cd = ssodam.title_tokenized.apply(lambda x: search_cd(word,x))
    text_cd = ssodam.text_tokenized.apply(lambda x: search_cd(word,x))
    comments_cd = ssodam.comments_tokenized.apply(lambda x: search_cd(word,x))
    comments_writer_cd = ssodam.comments_writer_tokenized.apply(lambda x: search_cd(word,x))
    ssodam = ssodam_month[title_cd | text_cd | comments_cd | comments_writer_cd]
    if show_urls==True:
        [print('http://www.ssodam.com/content/'+str(number)) for number in ssodam.num]
    return ssodam

In [31]:
s_search('충성', 6)

<ipython-input-14-3e0cb3931a1a>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df_month[title_cd | text_cd | comments_cd | comments_writer_cd]


http://www.ssodam.com/content/1178307
http://www.ssodam.com/content/1178860
http://www.ssodam.com/content/1180775
http://www.ssodam.com/content/1180776
http://www.ssodam.com/content/1181840
http://www.ssodam.com/content/1184360
http://www.ssodam.com/content/1187570
http://www.ssodam.com/content/1187956
http://www.ssodam.com/content/1189033
http://www.ssodam.com/content/1191402
http://www.ssodam.com/content/1193159
http://www.ssodam.com/content/1198158


,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized,text_w,title_w
63445,1178307,익게2,방탄소년단 빌보드 100 1위,", 에 이어 3번째피처링한 포함하면 4번째",익명,2021-06-02,02:19,3555.0,118.0,13.0,"와, 한국이랑 일본 음원사이트 퍼펙트 올킬 찍고, 미국 빌보드 핫백 핫샷데뷔랑 올킬...",,76.0,1,"[(이어, Verb), (3, Number), (번, Noun), (째, Suffi...","[(방탄소년단, Noun), (빌보드, Noun), (100, Number), (1...","[(한국, Noun), (일본, Noun), (음원, Noun), (사이트, Nou...",[],"[3, 번째, 피처링, 포함, 4번째]","[방탄소년단, 빌보드, 100, 1, 위]"
63832,1178860,익게2,안센 백신 30대 완판 이유 떴다,"그러나 슬프게도 가짜뉴스로 판명났다고 한다,,,",익명,2021-06-02,20:07,560.0,7.0,1.0,"군대 다녀온 것을 미국이 보상해줌 ㅋㅋㅋ 어메이징 코리아, 킹조국 충성충성, 갓조국...",,3.0,0,"[(슬프게도, Adjective), (가짜, Noun), (뉴스, Noun), (판...","[(센, Verb), (백신, Noun), (30, Number), (대, Verb...","[(군대, Noun), (다녀온, Verb), (미국, Noun), (보상, Nou...",[],"[가짜, 뉴스, 판명]","[백신, 30, 완판, 이유]"
65085,1180775,익게2,라붐 상상더하기 먼가 억지 역주행 만드는 느낌,"쁘걸 롤린은자연스럽게 역주행됐고, 롤린 인기에 탄력받아서 운전만해도 같이 역주행됐는...",익명,2021-06-05,22:08,5349.0,158.0,25.0,"ㄹㅇㅋㅋㅋㅋ, 상상더하기 겁나좋긴했는데 , 나도 예전부터 엄청 들었는데 막상 요즘은...","음 엄밀하게 사재기까진 아니긴한데 ㅋㅋ 비스꾸리무리하긴하지ㅋㅋ, ㅇㅇ 원래 아이유 ...",71.0,1,"[(쁘걸, Noun), (롤린, Noun), (자연, Noun), (스럽게, Jos...","[(라붐, Noun), (상상, Noun), (더하기, Noun), (먼가, Adj...","[(ㄹㅇㅋㅋㅋㅋ, KoreanParticle), (상상, Noun), (더하기, N...","[(음, Noun), (엄밀하게, Adjective), (사재기, Noun), (까...","[쁘걸, 롤린, 자연, 역주행, 롤린, 인기, 탄력, 운전, 만해, 역주행, 라붐,...","[라붐, 상상, 더하기, 억지, 역주행, 느낌]"
65086,1180776,연애상담소,19 자취방 영화보러 온 간략 후기,19지만 별건 없엉 ㅋㅋ나갈 일이 취소돼서 걍 얼른 끝낼게저번 글에서 용기를 준 다...,익명,2021-06-05,22:08,4208.0,136.0,63.0,ㅎㅓ 내가 무슨 부귀영화를 누리겠다고 연게에 들어와서 이걸 봤지ㅠㅠㅜㅠㅜㅠㅜㅠ 근데...,"뭔가 이렇게 설렘 가득한 ㅅㅅ 오랜만이어서 개좋았다, 넘 야설이될거같아서 ㅋㅋ 이정...",35.0,1,"[(19, Number), (별, Modifier), (건, Noun), (없, A...","[(19, Number), (자취, Noun), (방, Noun), (영화, Nou...","[(ㅎㅓ, KoreanParticle), (내, Noun), (부귀, Noun), ...","[(뭔가, Noun), (이렇게, Adverb), (설렘, Adjective), (...","[19, 별건, 일이취소, 얼른, 저번, 글, 용기, 준, 연애, 한지, 뭔가, 결...","[19, 자취, 방, 영화, 온, 간략, 후기]"
65768,1181840,익게2,삼성 충성 다미들 제품 뭐뭐써,"나는핏2, 20, 삼성노트북펜, 탭7, 버즈 프로 ...",익명,2021-06-07,16:37,290.0,1.0,1.0,"10 7 버즈프로 , 알짜배기만 씁니다 애플 제품도 몇개 있간한데, 나도 문제 없음...","버즈프로 귀 괜찮아 난 이상 없던데, 킹정",17.0,0,"[(나는핏, Noun), (2, Number), (20, Number), (삼성, ...","[(삼성, Noun), (충성, Noun), (제품, Noun), (뭐, Excla...","[(10, Number), (7, Number), (버즈, Noun), (프로, N...","[(버즈, Noun), (프로, Noun), (귀, Noun), (괜찮아, Adje...","[나는핏, 2, 20, 삼성, 노트북, 펜, 탭, 7, 버즈, 프로, 충성, 충성,...","[삼성, 충성, 제품]"
67325,1184360,익게2,신입다미 전화받기싫어ㅠ,입사한지 2달된 신입다미야 ...,익명,2021-06-10,19:24,506.0,24.0,1.0,ㄹㅇ ㅋㅋㅋ 그거 행정병으로 들어가서 일병 막 달아서 혼자 하기 시작할때 중대장 행...,ㅠㅠㅠㅠ진짜ㅠㅠㅠ인턴이랑 알바에서도 전화 업무가 주였어서 전화는 이제 익숙한데 옆사...,11.0,0,"[(입사, Noun), (한지, Noun), (2, Number), (달, Noun...","[(신입, Noun), (전화, Noun), (받기, Verb), (싫어, Adje...","[(ㄹㅇ, KoreanParticle), (ㅋㅋㅋ, KoreanParticle), ...","[(ㅠㅠㅠㅠ, KoreanParticle), (ㅠㅠㅠ, KoreanParticle)...","[입사, 한지, 2, 달, 신입, 말, 그대로, 막내, 전화, 그게, 전화, 자체,...","[신입, 전화]"
69463,1187570,익게2,실험 족보 부탁할때 뭐라고 해야함,다음학기는 좀 편하게 다니고 싶어서 아는 선배분한테 ...,익명,2021-06-15,01:16,215.0,3.0,0.0,"13, 안녕하세요 오랜만에 연락드립니다 다름이 아니라 제가 이번학기에 00 과목을 ...","형님 아니면 누님, 존경합니다 충성충성",5.0,0,"[(학기, Noun), (편하게, Adjective), (다니고, Verb), (싶...","[(실험, Noun), (족보, Noun), (부탁, Noun), (할, Verb)...","[(13, Number), (안녕하세요, Adjective), (오랜, Modifi...","[(형님, Noun), (누님, Noun), (존경, Noun), (합니다, Ver...","[학기, 선배분, 마지막, 전공, 실험, 족보, 말, 보은, 밥, 끼, 드릴, 예정...","[실험, 족보, 부탁, 함]"
69743,1187956,익게2,유통 이문제는 답 머했어,제조업체가 유통업체의 판매실적에만 집중해서 관계 관리하는 경우 발생하는 거,익명,2021-06-15,16:29,239.0,0.0,0.0,"유통비용 증가였나 몰겠다, 나도 유통비용 증가한듯 , 나듀, 나는 중간상에 대한 영...","나도 그거함 ㅎ, 나도 그거랑 고민하다가 그냥 유통비용 상승골랐어ㅜㅜ, 개망했네ㅋ...",11.0,0,"[(제조업체, Noun), (유통업체, Noun), (판매, Noun), (실적, ...","[(유통, Noun), (문제, Noun), (답, Noun), (머, Noun),...","[(유통, Noun), (비용, Noun), (증가, Noun), (였나, Verb...","[(나도, Verb), (그거, Noun), (함, Noun), (ㅎ, Korean...","[제조업체, 유통업체, 판매, 실적, 집중, 관계, 관리, 경우, 발생]","[유통, 문제, 답, 머]"
70495,1189033,익게2,맞선임 장기 이겼다 ㅎㅎ,왤케좋냐 ㅎㅎ,익명,2021-06-16,20:08,148.0,1.0,0.0,"그정도 충성장기 뒀는데도 지네, 이제 선임이 갈 때가 됐다, 사실상 패배","7수 물러줬고 첨에 불리

In [19]:
target = ssodam[ssodam['board'] == '맛집']
tokenized_text = [
    list(zip((sentence), (sentence)[1:]))
    for sentence in target.text_w.values
]  # text_W 토큰화

tokenized_title = [
    list(zip((sentence), (sentence)[1:]))
    for sentence in target.title_w.values
]  # title_w 토큰화해서 합치기

tokenized_nouns = tokenized_text + tokenized_title

counter = Counter()
for i in tokenized_nouns:
    counter.update(Counter(i))
mc = counter.most_common()
mc

[(('마카', '세'), 22),
 (('가성', '비'), 10),
 (('업종', '마카'), 7),
 (('츠', '마미'), 7),
 (('세', '주소'), 6),
 (('웨이', '팅'), 6),
 (('시라', '꼬'), 5),
 (('아귀', '간'), 5),
 (('경의선', '숲길'), 5),
 (('카츠', '카레'), 5),
 (('학교', '근처'), 4),
 (('자연', '산'), 4),
 (('이천', '호타루'), 4),
 (('연어', '밥'), 4),
 (('츠', '벤또'), 4),
 (('가격', '생각'), 4),
 (('학교', '주변'), 4),
 (('가츠', '벤또'), 4),
 (('짜장', '우동'), 4),
 (('재', '방문'), 4),
 (('쑥', '라떼'), 4),
 (('맛집', '추천'), 4),
 (('고이짱', '돈카츠'), 4),
 (('우동', '국물'), 4),
 (('4만원', '가격'), 4),
 (('코', '부지'), 4),
 (('꼭', '한번'), 3),
 (('1', '인'), 3),
 (('방어', '뱃살'), 3),
 (('스시', '찌'), 3),
 (('주소', '서울'), 3),
 (('주소', '서울특별시'), 3),
 (('밥', '조화'), 3),
 (('합정', '소토'), 3),
 (('아시', '분'), 3),
 (('성게', '알'), 3),
 (('알', '맛'), 3),
 (('숙성', '회'), 3),
 (('맛집', '기프티콘'), 3),
 (('기프티콘', '서비스'), 3),
 (('브로', '일링'), 3),
 (('일링', '커피'), 3),
 (('커피', '컴퍼니'), 3),
 (('그믐', '족발'), 3),
 (('창작', '촌'), 3),
 (('학교', '앞'), 3),
 (('친구', '데리'), 3),
 (('오전', '11시'), 3),
 (('카레', '9000원'), 3),
 (('튀김', '옷'), 3),
 (('앞'